In [1]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
import os
os.putenv('SDL_VIDEODRIVER', 'fbcon')
os.environ["SDL_VIDEODRIVER"] = "dummy"  # this line make pop-out window not appear
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
import numpy as np
import skimage.color
import skimage.transform
from tqdm import tqdm

In [2]:
!rm -rf ./movie && mkdir ./movie

In [3]:
from ple import PLE
from ple.games.flappybird import FlappyBird

game = FlappyBird()
env = PLE(game, fps=30, display_screen=False)  # environment interface to game

screen_width = 80
screen_height = 80
num_stack = 4

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [4]:
class Actor_critic:
    def __init__(self, name, num_action, discount_factor=0.99):
        self.discount_factor = discount_factor
        self.num_action = num_action
        self.name = name
        with tf.variable_scope(name):
            self.build_model()

    def build_model(self):
        # input: current screen, selected action and reward
        self.input_screen = tf.placeholder(
            tf.float32, shape=[None, num_stack, 5])
        self.action = tf.placeholder(tf.int32, [None])
        self.reward = tf.placeholder(tf.float32, [None])
        self.is_training = tf.placeholder(tf.bool, shape=[])

        def value_net(screen, reuse=False):
            with tf.variable_scope(
                    "value_net",
                    reuse=reuse,
                    initializer=tf.truncated_normal_initializer(stddev=1e-2)):
                x = tf.contrib.layers.flatten(screen)
                x = tf.layers.dense(x, units=512, activation=tf.nn.relu)
                x = tf.layers.dense(x, units=512, activation=tf.nn.relu)
                x = tf.layers.dense(x, units=512, activation=tf.nn.relu)
                x = tf.layers.dense(x, units=512, activation=tf.nn.relu)
                x = tf.layers.dense(x, units=1, activation=None)
                return x

        def policy_net(screen, reuse=False):
            with tf.variable_scope("policy_net", reuse=reuse):
                x = tf.contrib.layers.flatten(screen)
                x = tf.layers.dense(x, units=512, activation=tf.nn.relu)
                x = tf.layers.dense(x, units=512, activation=tf.nn.relu)
                x = tf.layers.dense(x, units=512, activation=tf.nn.relu)
                x = tf.layers.dense(x, units=512, activation=tf.nn.relu)
                x = tf.layers.dense(x, units=self.num_action, activation=None)
                return x

        # value
        self.v_output = value_net(
            self.input_screen
        )  # Q(s,a,theta) for all a, shape (batch_size, num_action)
        self.tar_V = tf.placeholder(tf.float32, [None])
        self.V_loss = tf.reduce_mean(
            tf.square(self.reward + self.discount_factor * self.tar_V -
                      self.v_output))
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-6)
        g_gvs = optimizer.compute_gradients(
            self.V_loss,
            var_list=[
                v for v in tf.global_variables() if 'value_net' in v.name
            ])
        self.V_train_op = optimizer.apply_gradients(g_gvs)

        # policy
        self.policy_logit = policy_net(
            self.input_screen
        )  # logit of probility(P(s,a,theta)) for all a, shape (batch_size, num_action)
        index = tf.stack([tf.range(tf.shape(self.action)[0]), self.action],
                         axis=1)
        self.prob = tf.gather_nd(
            tf.nn.softmax(self.policy_logit),
            index)  # P(s,a,theta) for selected action, shape (batch_size, 1)

        # loss = E[log(p(s,a))*r]
        self.policy_loss = -tf.reduce_mean(
            tf.log(self.prob + 0.00000001) * self.reward)
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-6)
        g_gvs = optimizer.compute_gradients(
            self.policy_loss,
            var_list=[
                v for v in tf.global_variables() if 'policy_net' in v.name
            ])
        self.train_op = optimizer.apply_gradients(g_gvs)
        self.pred = tf.multinomial(self.policy_logit,
                                   1)  # sample action from distribution

    def select_action(self, input_screen, sess):
        input_screen = np.array(input_screen)
        feed_dict = {
            self.input_screen: input_screen[None, :],
        }
        action = sess.run(
            self.pred,
            feed_dict=feed_dict)[0][0]  # sameple action from distribution
        return action

    def update_policy(self, input_screens, actions, rewards,
                      input_screens_plum):
        feed_dict = {
            self.input_screen:
            np.array(input_screens_plum)
        }
        esti_V = sess.run(self.v_output, feed_dict=feed_dict).flatten()
        td_target = rewards + self.discount_factor * esti_V

        feed_dict = {
            self.input_screen: np.array(input_screens),
        }
        esti_V = sess.run(self.v_output, feed_dict=feed_dict).flatten()
        td_error = td_target - esti_V
        feed_dict = {
            self.input_screen:
            np.array(input_screens_plum),
        }
        feed_dict = {
            self.input_screen: np.array(input_screens),
            self.tar_V: td_target,
            self.reward: rewards,
        }

        V_loss, _ = sess.run([self.V_loss, self.V_train_op],
                             feed_dict=feed_dict)

        feed_dict = {
            self.input_screen: np.array(input_screens),
            self.action: actions,
            self.reward: td_error,
        }
        policy_loss, _ = sess.run([self.policy_loss, self.train_op],
                                  feed_dict=feed_dict)
        return V_loss, policy_loss


In [5]:
# init agent
tf.reset_default_graph()
num_action = len(env.getActionSet())
# agent for frequently updating
ac_agent = Actor_critic('PG_Agent', num_action)
# init all
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [6]:
def preprocess(state):
    data = [
        state['player_y'],
        state['player_vel'],
        state['next_pipe_top_y'],
        state['next_pipe_bottom_y'],
        state['next_pipe_dist_to_player'],
#         state['next_next_pipe_top_y'],
#         state['next_next_pipe_bottom_y'],
#         state['next_next_pipe_dist_to_player'],
    ]
    
    return data

def make_anim(images, fps=60, true_image=False):
    duration = len(images) / fps
    import moviepy.editor as mpy

    def make_frame(t):
        try:
            x = images[int(len(images) / duration * t)]
        except:
            x = images[-1]

        if true_image:
            return x.astype(np.uint8)
        else:
            return ((x + 1) / 2 * 255).astype(np.uint8)

    clip = mpy.VideoClip(make_frame, duration=duration)
    clip.fps = fps
    return clip

In [7]:
from IPython.display import Image, display

update_every_episode = 1
print_every_episode = 100
save_video_every_episode = 500
NUM_EPISODE = 30_000
NUM_EXPLORE = 0
reward_values = {
    "positive": 1,
    "tick": 0.2,  # reward per timestamp
    "loss": -1,
}
for episode in range(0, NUM_EPISODE + 1):

    # Reset the environment
    game = FlappyBird()
    env = PLE(
        game,
        fps=30,
        display_screen=False,
        reward_values=reward_values,
        rng=np.random.RandomState(1))
    env.reset_game()
    env.act(0)  # dummy input to make sure input screen is correct

    # record frame
    if episode % save_video_every_episode == 0:
        frames = [env.getScreenRGB()]

    # grayscale input screen for this episode
#     input_screens = [preprocess(env.getScreenGrayscale())] * 4
    input_screens = [preprocess(game.getGameState())] * 4

    # cumulate reward for this episode
    cum_reward = 0

    experiences = []
    t = 0
    while not env.game_over():
        # feed four previous screen, select an action
        action = ac_agent.select_action(input_screens[-4:], sess)

        # execute the action and get reward
        reward = env.act(env.getActionSet()[action])

        # record frame
        if episode % save_video_every_episode == 0:
            frames.append(env.getScreenRGB())

        # cumulate reward
        cum_reward += reward

        # append grayscale screen for this episode
        input_screens.append(preprocess(game.getGameState()))

        # append experience for this episode
        experiences.append(
            [input_screens[-5:-1], action, reward, input_screens[-4:]])

        t += 1

    def discount_reward(x, discount_rate):
        discounted_r = np.zeros(len(x))
        num_r = len(x)
        for i in range(num_r):
            discounted_r[i] = x[i] * math.pow(discount_rate, i)
        discounted_r = np.cumsum(discounted_r[::-1])
        return discounted_r[::-1]

    rewards = [e[2] for e in experiences]
    discounted_reward = discount_reward(rewards, ac_agent.discount_factor)

    # normalize
    discounted_reward -= np.mean(discounted_reward)
    discounted_reward /= np.std(discounted_reward)
    train_screens = []
    train_actions = []
    train_rewards = []
    train_input_screens_plum = []
    for i in range(len(experiences)):
        experiences[i][2] = discounted_reward[i]
        train_screens.append(experiences[i][0])
        train_actions.append(experiences[i][1])
        train_rewards.append(experiences[i][2])
        train_input_screens_plum.append(experiences[i][3])
    loss = ac_agent.update_policy(train_screens, train_actions, train_rewards,
                                  train_input_screens_plum)

    if episode % print_every_episode == 0 and episode > NUM_EXPLORE:
        print("[{}] time live:{}, cumulated reward: {:.5f}, loss: {}".format(
            episode, t, cum_reward, loss))

    if episode % save_video_every_episode == 0 and episode > NUM_EXPLORE:  # for every 5000 episode, record an animation
        clip = make_anim(frames, fps=60, true_image=True).rotate(-90)
        clip.write_videofile("movie/ac_{}_{}.webm".format(episode, t), fps=60)
#         display(clip.ipython_display(fps=60, autoplay=1, loop=1))


[100] time live:61, cumulated reward: 11.20000, loss: (3.9600792, -0.0011545287)
[200] time live:61, cumulated reward: 11.20000, loss: (3.9600897, -0.0005060394)
[300] time live:61, cumulated reward: 11.20000, loss: (3.9600933, -0.00046241612)
[400] time live:61, cumulated reward: 11.20000, loss: (3.9600933, -0.00079019374)
[500] time live:61, cumulated reward: 11.20000, loss: (3.9600577, -0.07273477)
[MoviePy] >>>> Building video movie/ac_500_61.webm
[MoviePy] Writing video movie/ac_500_61.webm


100%|██████████| 63/63 [00:00<00:00, 117.52it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/ac_500_61.webm 



[600] time live:61, cumulated reward: 11.20000, loss: (3.9600933, -0.0017635616)
[700] time live:61, cumulated reward: 11.20000, loss: (3.960096, -0.00078133814)
[800] time live:61, cumulated reward: 11.20000, loss: (3.9600937, -0.0014968382)
[900] time live:61, cumulated reward: 11.20000, loss: (3.9600973, -0.0011133823)
[1000] time live:61, cumulated reward: 11.20000, loss: (3.9600897, -0.003660325)
[MoviePy] >>>> Building video movie/ac_1000_61.webm
[MoviePy] Writing video movie/ac_1000_61.webm


100%|██████████| 63/63 [00:00<00:00, 120.33it/s]

...

100%|██████████| 587/587 [00:04<00:00, 118.89it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/ac_29500_585.webm 



[29600] time live:2434, cumulated reward: 548.80000, loss: (3.9601, 0.0024035084)
[29700] time live:2434, cumulated reward: 548.80000, loss: (3.9601, -0.003987331)
[29800] time live:2470, cumulated reward: 557.00000, loss: (3.9601007, -0.008694058)
[29900] time live:811, cumulated reward: 181.20000, loss: (3.960101, -0.022795625)
[30000] time live:585, cumulated reward: 130.00000, loss: (3.9600995, -8.446966e-05)
[MoviePy] >>>> Building video movie/ac_30000_585.webm
[MoviePy] Writing video movie/ac_30000_585.webm


100%|██████████| 587/587 [00:05<00:00, 117.06it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie/ac_30000_585.webm 



# Report 

I found it difficult to train. Tried a lot of different hyper parameters and didn't find a stable combination. Maybe it's the nature of RL: unable to reproduce the result. I've get a time live 2470!!! I think it's the upp

I have spent 2 days entirely for tweaking the hyper parameters...I have question about the exploration part. The code has `exploration ratio` but not using it at all. This seems weired...

Comparing with the CNN one, IMO, it's more diffult to make the model converge. I use only the info of next-pipe, no next-next pipe. According to the visualization, I think I've achieve the upper bound of using such data. The place where the bird fails is a sudden change of pipe position.

In [17]:
from IPython.display import HTML
from base64 import b64encode

with open('./movie/ac_29000_2470.webm', 'rb') as f:
    b64 = b64encode(f.read()).decode()
HTML(data='''<video controls> 
<source type="video/webm" src="data:video/webm;base64,{}">
</video>
'''.format(b64))